In [3]:
import pandas as pd
import numpy as np

## DATA CLEANING AND WRITING [DO NOT RUN THIS ANYMORE]

In [ ]:
#Data Cleaning Notebook
#Load speaker attributes parquet file
dfattrib = pd.read_parquet("/content/drive/MyDrive/ADA/speaker_attributes.parquet/speaker_attributes.parquet")

#Load QID correspondance file and remove description column
dflabels = pd.read_csv('/content/drive/MyDrive/ADA/wikidata_labels_descriptions_quotebank.csv.bz2', compression='bz2', index_col = 'QID')
dflabels = dflabels.drop(['Description'], axis=1)


In [ ]:
#Drop
dfattrib = dfattrib.drop(['aliases','lastrevid','US_congress_bio_ID','party','candidacy','type','label','ethnic_group'], axis=1) #Remove

In [ ]:
#Extract the first element only
dfattrib["date_of_birth"] = dfattrib["date_of_birth"].apply(lambda x: x[0] if x is not None else x) 
dfattrib["gender"] = dfattrib["gender"].apply(lambda x: x[0] if x is not None else x) 
dfattrib["occupation"] = dfattrib["occupation"].apply(lambda x: x[0] if x is not None else x)
dfattrib["nationality"] = dfattrib["nationality"].apply(lambda x: x[0] if x is not None else x) 
dfattrib["religion"] = dfattrib["religion"].apply(lambda x: x[0] if x is not None else x) 
dfattrib["academic_degree"] = dfattrib["academic_degree"].apply(lambda x: x[0] if x is not None else x) 

In [ ]:
#Missing labels
Missing =  ['Q6296256','Q3268166','Q11815360','Q12014399','Q16287483','Q20432251','Q21550646','Q13365117','Q13424794','Q1248362','Q3186984','Q6859927','Q15145782','Q15991263','Q99753484','Q12455619','Q5568256','Q6363085','Q11819457','Q11819457','Q12334852','Q15145783','Q11815360']

In [ ]:
#Replace missing
dfattrib["occupation"] = dfattrib["occupation"].apply(lambda x: None if x in Missing else x) 
dfattrib["gender"] = dfattrib["gender"].apply(lambda x: None if x in Missing else x)
dfattrib["religion"] = dfattrib["religion"].apply(lambda x: None if x in Missing else x) 
dfattrib["nationality"] = dfattrib["nationality"].apply(lambda x: None if x in Missing else x) 
dfattrib["academic_degree"] = dfattrib["academic_degree"].apply(lambda x: None if x in Missing else x) 

In [ ]:
#Change QIDs
dfattrib["gender"] = dfattrib["gender"].apply(lambda x: dflabels.loc[x].item() if x is not None else x) 
dfattrib["nationality"] = dfattrib["nationality"].apply(lambda x: dflabels.loc[x].item() if x is not None else x)  
dfattrib["religion"] = dfattrib["religion"].apply(lambda x: dflabels.loc[x].item() if x is not None else x)  
dfattrib["occupation"] = dfattrib["occupation"].apply(lambda x: dflabels.loc[x].item() if x is not None else x)
dfattrib["academic_degree"] = dfattrib["academic_degree"].apply(lambda x: dflabels.loc[x].item() if x is not None else x)   

In [ ]:
dfattrib = dfattrib.set_index('id')

In [ ]:
#Clean Date of Birth
dfattrib["date_of_birth"] = dfattrib["date_of_birth"].astype('string')
birth_outlier = dfattrib["date_of_birth"].iloc[4555848]
dfattrib["date_of_birth"] = dfattrib["date_of_birth"].apply(lambda x: -9999 if not isinstance(x, str) else -9999 if x == birth_outlier else x)
dfattrib["date_of_birth"] = dfattrib["date_of_birth"].apply(lambda x: int(x[1:5]) if isinstance(x, str)  else -9999)
dfattrib["date_of_birth"] = dfattrib["date_of_birth"].astype(float)
dfattrib = dfattrib[(dfattrib["date_of_birth"]>1928) & (dfattrib["date_of_birth"]<2016)]

In [ ]:
dfattrib.to_csv("/content/drive/MyDrive/ADA/df_speaker_raw.csv.gzip", header=True, index=True, compression="gzip")

## Processing

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ADA/df_speaker_raw.csv.gzip', compression="gzip")

In [ ]:
#Fix genders
Male = ['male']
Female = ['female']
All = Male + Female
df["gender"] = df["gender"].apply(lambda x: x if x is not None else x)
df["gender"] = df["gender"].apply(lambda x: 'Male' if x in Male else 'Female' if x in Female else 'Other' if (x not in All and not None) else x)

In [ ]:
#Fix academic
Master = ['master', 'masters']
Bachelor = ['bachelor', 'bachelors']
Doctorate = ['doctorate','doctor','doktor']
All = ['master', 'masters', 'bachelor', 'bachelors', 'doctorate','doctor','doktor']
df["academic_degree"] = df["academic_degree"].apply(lambda x: None if not isinstance(x,str) else 'Master' if any(ext in x.lower() for ext in Master) else 'Bachelor' if any(ext in x.lower() for ext in Bachelor) else 'Doctorate' if any(ext in x.lower() for ext in Doctorate) else 'Other' if not any(ext in x.lower() for ext in All) else x)

In [ ]:
#Fix religion
Christians = ['church', 'luther', 'anglican', 'catho', 'protest', 'presby', 'calvinism', 'purita', 'christ']
Muslims = ['sunni', 'chiisme', 'shia', 'islam', 'shiisme', 'druz']
Jewish = ['jewish','synago','jew', 'hebrew', 'juda', 'hebr', 'israel', 'yiddish']
Hindus = ['hind', 'budd', 'shiva']
All = ['church', 'luther', 'anglican', 'catho', 'protest', 'presby', 'calvinism', 'purita', 'christ', 'sunni', 'chiisme', 'shia', 'islam', 'shiisme', 'druz', 'jewish','synago','jew', 'hebrew', 'juda', 'hebr', 'israel', 'yiddish', 'hind', 'budd', 'shiva']
df["religion"] = df["religion"].apply(lambda x: None if not isinstance(x,str) else 'Christian' if any(ext in x.lower() for ext in Christians) else 'Muslim' if any(ext in x.lower() for ext in Muslims) else 'Jewish' if any(ext in x.lower() for ext in Jewish) else 'Hindus' if any(ext in x.lower() for ext in Hindus) else 'Other' if not any(ext in x.lower() for ext in All) else x)

In [ ]:
#Fix academic
Sports = ['coach','ball','athl','weight','sport','player','swimmer','skier','racing']
Arts = ['actor','director','author','pres','anch','arch','art','danc','sing','act','paint','composer','pianist','violinist','guitarist','drummer','music','art','write','wright','comed','film','poet','chore','graph','edit','prod','sculpt','hist','geo','journ','law','judge','speak','sax','sing','dance','draw','book','lyric','design']
Sciences = ['novel','philo','scien','research','engin','tech','med','nurse','phys','chem','mech','elec','psych','prof','math','bio','schol','surgeon','lab','botan','teacher','university']
Politics = ['dipl','polit','ruler','minister','dep','president','gov']
Military = ['sold','milit','gener','armed','army','war','force']
Religion = ['catho','priest','imam','church','buddh','pastor','mission','theolog','clergy','christian','muslim','jewish','rabbi','monk']
Business = ['econ','business','entrepr','manager','exec']
All = Sports + Arts + Sciences + Politics + Military + Religion + Business
df["occupation"] = df["occupation"].apply(lambda x: None if not isinstance(x,str) else 'Sports' if any(ext in x.lower() for ext in Sports) else 'Arts' if any(ext in x.lower() for ext in Arts) else 'Politics' if any(ext in x.lower() for ext in Politics) else 'Military' if any(ext in x.lower() for ext in Military) else 'Sciences' if any(ext in x.lower() for ext in Sciences) else 'Religion' if any(ext in x.lower() for ext in Religion) else 'Business' if any(ext in x.lower() for ext in Business) else 'Other' if not any(ext in x.lower() for ext in All) else x)

In [ ]:
df.head(5)

,id,date_of_birth,nationality,gender,occupation,academic_degree,religion
0,Q42,1952.0,United Kingdom,Male,Arts,None,None
1,Q207,1946.0,United States of America,Male,Politics,None,Christian
2,Q633,1945.0,Canada,Male,Arts,None,None
3,Q640,1969.0,Germany,Male,Arts,None,None
4,Q853,1932.0,Soviet Union,Male,Arts,None,Christian


In [ ]:
df.to_csv("/content/drive/MyDrive/ADA/df_speaker_cooked.csv.gzip", header=True, index=True, compression="gzip")

### Check the output file

In [4]:
df_cooked = pd.read_csv('/content/drive/MyDrive/ADA/df_speaker_cooked.csv.gzip', compression="gzip")
df_raw = pd.read_csv('/content/drive/MyDrive/ADA/df_speaker_raw.csv.gzip', compression="gzip")

In [5]:
df_raw.head(5)

,id,date_of_birth,nationality,gender,occupation,academic_degree,religion
0,Q42,1952.0,United Kingdom,male,playwright,NaN,NaN
1,Q207,1946.0,United States of America,male,politician,NaN,United Methodist Church
2,Q633,1945.0,Canada,male,singer,NaN,NaN
3,Q640,1969.0,Germany,male,photographer,NaN,NaN
4,Q853,1932.0,Soviet Union,male,film director,NaN,Eastern Orthodox Church


In [6]:
df_cooked.head(5)

,Unnamed: 0,id,date_of_birth,nationality,gender,occupation,academic_degree,religion
0,0,Q42,1952.0,United Kingdom,Male,Arts,NaN,NaN
1,1,Q207,1946.0,United States of America,Male,Politics,NaN,Christian
2,2,Q633,1945.0,Canada,Male,Arts,NaN,NaN
3,3,Q640,1969.0,Germany,Male,Arts,NaN,NaN
4,4,Q853,1932.0,Soviet Union,Male,Arts,NaN,Christian
